In [1]:
import pandas as pd
import seaborn as sns
import string
import warnings
import matplotlib.pyplot as plt
import pickle
import numpy as np
import spacy 
from spacy.lang.en import English
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")

#### 1. Read in data

In [2]:
df = pd.read_csv('/home/jupyter/sb-entity-classification/data/data.csv')
df.columns = ['class','name']

# classes_list = pd.read_csv('/home/jupyter/sb-entity-classification/data/classes.txt', header = None)
# classes_list['class'] = classes_list.index
# classes_list.columns = ['class_name', 'class']
# classes_list['class'] = classes_list['class'] + 1  # based on information provided in the brief

# df = df.merge(classes_list, on = 'class', how = 'left')

####  2. Train, Validation, Test split

In [3]:
train, test = train_test_split(df, test_size = 0.2, random_state = 42, shuffle = True)

In [4]:
train.shape

(439029, 2)

In [5]:
train = train.sample(frac = 0.1)

In [6]:
train.shape

(43903, 2)

In [7]:
### save the split train, valid, and test set, for fair comparison between models
# train.to_pickle('/home/jupyter/sb-entity-classification/data/train.pkl')
# test.to_pickle('/home/jupyter/sb-entity-classification/data/test.pkl')

#### 3. Create features

##### 3.1. Simple statistical features already explored in EDA

In [8]:
def create_statistical_features(df):
    df_features = df.copy()

    df_features['num_chars'] = df['name'].str.len()
    df_features['num_words'] = df_features['name'].apply(lambda x: len(x.split()))

    count = lambda l1,l2: sum([1 for x in l1 if x in l2]) 
    df_features['num_punctuations']= df_features['name'].apply(lambda x: count(x,set(string.punctuation)))
    df_features['num_ascii']= df_features['name'].apply(lambda x: count(x,set(string.ascii_letters)))
    df_features['num_whitespace']= df_features['name'].apply(lambda x: count(x,set(string.whitespace)))
    df_features['num_digits']= df_features['name'].apply(lambda x: count(x,set(string.digits)))
    df_features['num_nonascii'] = df_features['num_chars']-df_features['num_punctuations']-df_features['num_digits']-df_features['num_ascii']- df_features['num_whitespace']
    df_features = df_features.drop(['num_whitespace', 'num_ascii'], axis = 1)
    
    return df_features

In [9]:
train_statistical_features = create_statistical_features(train)
all_cols = train_statistical_features.columns.tolist()
statistical_feature_cols = [x for x in all_cols if x not in ['class', 'name']]
train_statistical_features = train_statistical_features[statistical_feature_cols]

##### 3.2 keyword in brackets also explored in EDA

In [10]:
# based on common sense -
hand_code_keywords = ['company']+['musician', 'singer', 'writer', 'artist', 'author']\
+ ['footballer', 'cricketer', 'football', 'baseball', 'rugby', 'hockey'] +\
['politician'] +['ship'] + ['crater', 'river'] + ['horse'] +['plant']\
+['album', 'EP', 'soundtrack'] +['film'] +['novel', 'magazine', 'book', 'journal', 'play', 'comics', 'newspaper', 'manga']

In [11]:
def create_keyword_features(df):
    df_features = df.copy()
    df_features['within_brackets'] = df['name'].str.extract('.*\((.*)\).*')

    df_features_with_brackets = df_features[['within_brackets']].dropna(axis = 0)
    for keyword in hand_code_keywords:
        df_features_with_brackets['has_{}'.format(keyword)] = df_features_with_brackets['within_brackets']\
                                                                .apply(lambda x: keyword in x.lower() )

    df_features = df_features.drop('within_brackets', axis = 1)
    df_features_with_brackets = df_features_with_brackets.drop('within_brackets', axis = 1)

    df_features_with_brackets.replace({True:1, False:0}, inplace = True)
    df_features = df_features.merge(df_features_with_brackets, left_index = True, right_index = True, how = 'left')
    df_features.fillna(0, inplace = True)
    
    return df_features

In [12]:
train_keyword_features = create_keyword_features(train)
all_cols = train_keyword_features.columns.tolist()
keyword_feature_cols = [x for x in all_cols if x not in ['class', 'name']]
train_keyword_features = train_keyword_features[keyword_feature_cols]

##### 3.3 NER explored in EDA

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
def create_ner_features(df, NER_labels = None):
    
    df_features = df.copy()

    df_features['NER_labels'] = df_features['name'].apply(lambda x: [i.label_ for i in nlp(x).ents])
    df_features_NER = df_features[df_features['NER_labels'].apply(lambda x: len(x) !=0)][['NER_labels']]
    
    if NER_labels == None:
        all_NER_labels = set(np.concatenate(df_features_NER['NER_labels'].tolist()))
    else:
        all_NER_labels = NER_labels
        
    for ner in all_NER_labels:
        df_features_NER['ner_{}'.format(ner)] = df_features_NER['NER_labels'].apply(lambda x: ner in x )

    df_features = df_features.drop('NER_labels', axis = 1)
    df_features_NER = df_features_NER.drop('NER_labels', axis = 1)

    df_features_NER.replace({True:1, False:0}, inplace = True)
    df_features = df_features.merge(df_features_NER, left_index = True, right_index = True, how = 'left')
    df_features.fillna(0, inplace = True)
    
    return df_features, all_NER_labels

In [14]:
%%time
train_ner_features, NER_labels_trained = create_ner_features(train)
all_cols = train_ner_features.columns.tolist()
ner_feature_cols = [x for x in all_cols if x not in ['class', 'name']]
train_ner_features = train_ner_features[ner_feature_cols]

CPU times: user 4min 11s, sys: 113 ms, total: 4min 11s
Wall time: 4min 11s


#### 4. Prepare feature set

In [17]:
train = train.merge(train_statistical_features,left_index = True, right_index = True, how = 'left')\
.merge(train_keyword_features,left_index = True, right_index = True, how = 'left')

In [18]:
X = train.drop(['name','class'], axis = 1)
y = train['class']

#### 5. Train xgboost

In [19]:
model = XGBClassifier(nthread = 3)
model.fit(X, y)

[04:01:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=3, nthread=3, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

#### 6. Evaluate model

In [20]:
test = test.sample(frac = 0.1)

test_statistical_features = create_statistical_features(test)[statistical_feature_cols]
test_keyword_features = create_keyword_features(test)[keyword_feature_cols]

# test_ner_features, _ = create_ner_features(test, NER_labels_trained)
# test_ner_features = test_ner_features[ner_feature_cols]

In [21]:
test = test.merge(test_statistical_features,left_index = True, right_index = True, how = 'left')\
            .merge(test_keyword_features,left_index = True, right_index = True, how = 'left')

In [22]:
test.fillna(0, inplace = True)

In [23]:
X_test = test.drop(['name','class'], axis = 1)
y_test = test['class']

In [24]:
X_test

,num_chars,num_words,num_punctuations,num_digits,num_nonascii,has_company,has_musician,has_singer,has_writer,has_artist,...,has_soundtrack,has_film,has_novel,has_magazine,has_book,has_journal,has_play,has_comics,has_newspaper,has_manga
378736,23,2,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372839,22,2,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141169,16,2,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
264620,46,8,2,4,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66919,16,2,2,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366757,22,2,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7476,37,4,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140428,11,2,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
345517,17,2,1,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
y_pred = model.predict(X_test)

In [26]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 37.65%


In [27]:
from sklearn.metrics import classification_report

In [28]:
print(classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           1      0.471     0.052     0.093       793
           2      0.367     0.651     0.469       837
           3      0.254     0.273     0.263       846
           4      0.222     0.564     0.318       798
           5      0.579     0.200     0.298       804
           6      0.810     0.655     0.725       821
           7      0.267     0.218     0.240       771
           8      0.441     0.213     0.288       811
           9      0.348     0.471     0.400       798
          10      0.374     0.210     0.269       810
          11      0.321     0.612     0.421       786
          12      0.650     0.446     0.529       766
          13      0.444     0.464     0.454       776
          14      0.740     0.174     0.281       559

    accuracy                          0.377     10976
   macro avg      0.449     0.372     0.361     10976
weighted avg      0.443     0.377     0.363     10976



In [48]:
classes_list = pd.read_csv('/home/jupyter/sb-entity-classification/data/classes.txt', header = None)
classes_list['class'] = classes_list.index
classes_list.columns = ['class_name', 'class']
classes_list['class'] = classes_list['class'] + 1  # based on information provided in the brief
classes_list

,class_name,class
0,Company,1
1,EducationalInstitution,2
2,Artist,3
3,Athlete,4
4,OfficeHolder,5
5,MeanOfTransportation,6
6,Building,7
7,NaturalPlace,8
8,Village,9
9,Animal,10
